In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import MDS
from PIL import Image
import os

import ecoset
import categorization as cat
import utils

2023-05-14 23:14:01.914623: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 23:14:02.551956: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-14 23:14:02.552007: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-14 23:14:02.552014: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

# Dataset
100 images from Ecoset + CUB200.
* Superordinate: 50 vehicles and 50 animals
* Basic: 25 cars, 25 busses, 25 dogs, 25 birds
* Subordinate: 5 species of birds

Images preprocessed by centering the features and resizing to 224x224.

In [2]:
# Load csv
df = pd.read_csv("./deepCatsImages.csv")
df.head()


,path,name,cat1,cat2,cat3
0,./images/deepCats/animal/bird/CUB_005.Crested_...,Crested_Auklet_0021_794938.jpg,animal,bird,CUB_005.Crested_Auklet
1,./images/deepCats/animal/bird/CUB_005.Crested_...,Crested_Auklet_0028_794951.jpg,animal,bird,CUB_005.Crested_Auklet
2,./images/deepCats/animal/bird/CUB_005.Crested_...,Crested_Auklet_0036_794905.jpg,animal,bird,CUB_005.Crested_Auklet
3,./images/deepCats/animal/bird/CUB_005.Crested_...,Crested_Auklet_0011_794927.jpg,animal,bird,CUB_005.Crested_Auklet
4,./images/deepCats/animal/bird/CUB_005.Crested_...,Crested_Auklet_0006_1813.jpg,animal,bird,CUB_005.Crested_Auklet


In [6]:
imgFile = "./images/deepCatsImages.npy"

if not os.path.exists(imgFile):
    # Preallocate array for images
    images = np.zeros((df.shape[0], 224, 224, 3), dtype=np.float32)

    # Loop through images
    for i, row in df.iterrows():
        # Load image
        img = Image.open(row["path"])

        # Preprocess image
        img = ecoset.preprocess_alexnet(img)

        # Add image to array
        images[i] = img

    # Save images as a npy
    np.save(imgFile, images)
else:
    # Load images from npy
    print("Loading images from file...")
    images = np.load(imgFile)

print(images.shape)

Loading images from file...
(100, 224, 224, 3)


# Models and representations
Generate representations from AlexNet trained in Mehrer et al 2022. 
* 10 models are traiend on Ecoset (basic-level categories)
* 10 models are traiend on ImageNet (mixed levels of abstraction)